# Title

[Free Form Description]

**Resources**

- [Spotipy docs](https://spotipy.readthedocs.io/en/latest/)


**Data Sources:**

- `xxx.csv`: blablabla

**Data Output:**

- `xxx.csv`: blablabla

**Changes**

- 2019-02-18: Start project



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries,-load-data" data-toc-modified-id="Import-libraries,-load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries, load data</a></span></li></ul></div>

---

## Import libraries, instantiate client

In [70]:
# Import libraries
# import requests
# import json
from pprint import pprint
import json
import time
import numpy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import credentials  # file where secret discogs credentials are stored
# # My functions
# import EDA_functions as EDA
# import cleaning_functions as cleaning

# # Visualization
# import matplotlib.pyplot as plt
# import seaborn as sns 
# # sns.set_style('whitegrid')
# color = 'rebeccapurple'
# %matplotlib inline

# # Display settings
# from IPython.display import display
# pd.options.display.max_columns = 100

In [2]:
# Import credentials and instantiate client with authorization

SPOTIPY_CLIENT_ID = credentials.client_id
SPOTIPY_CLIENT_SECRET = credentials.client_secret

client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Request data

### Artist

In [3]:
# Get Artist URI

name = "Cannibal Corpse"

def get_artist_uri(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    artist_uri = items[0]['uri'] 
    return artist_uri

In [4]:
artist_uri = get_artist_uri(name)
pprint(artist_uri)

'spotify:artist:0yLwGBQiBqhXOvmTfH2A7n'


### Trackslist

The easiest way to query for tracks is as follows:

```python
results = sp.search(q=artist, limit=50, type='track')
for i, t in enumerate(results['tracks']['items']):
    print(' ', i, t['name'])
```

But problem is that the upper limit possible is 50, and CC have released many more songs than that, so I will try a work around. Get a list of all albums, clean it a bit and then combine all the tracks of each single album in the list.

In [5]:
# Get Artist albums (dict)
# Note: setting title as key catches some duplicates

def get_artist_albums(artist_uri):
    albums = {}
    results = sp.artist_albums(artist_uri, album_type='album')
    for i, item in enumerate(results['items']):
        albums[item['name'].title()] = item['uri']
    return albums

In [6]:
artist_albums = get_artist_albums(artist_uri)
pprint(artist_albums)

{'A Skeletal Domain': 'spotify:album:3Am2ExqPJjrw51AU2HRJSF',
 'Bloodthirst': 'spotify:album:4AObEzsEDC6WOAlJCRL9Pa',
 'Evisceration Plague': 'spotify:album:03jcqKD4ZthRxviJQIPCYf',
 'Gallery Of Suicide': 'spotify:album:3fEMj3Nr6Dpu7rFvgi1rg9',
 'Global Evisceration': 'spotify:album:4OZFh833qdZrK6DyelNAwS',
 'Gore Obsessed': 'spotify:album:4QnPTzLdkNwnvXxjKZ3AgF',
 'Kill': 'spotify:album:6ilBwRWtEedIQu4Q0YyIux',
 'Live Cannibalism': 'spotify:album:4eqhYFJAudlSc42bj0JKa9',
 'Red Before Black': 'spotify:album:2Ecgc9nNxJpXd3gboFPbvz',
 'The Bleeding': 'spotify:album:1FOBUa0z9RhKPH0frnWMRa',
 'The Bleeding - Reissue': 'spotify:album:18byNZK0g3PGvvebCYQpwm',
 'The Wretched Spawn': 'spotify:album:46QEzkj6cg7FxmIHL7zTjR',
 'Tomb Of The Mutilated': 'spotify:album:2yPAe2RJCfrfV97c8FKKjb',
 'Torture': 'spotify:album:7AaZ6zhZXDKilXqsJ52g8b',
 'Torturing And Eviscerating': 'spotify:album:2RInJ5yUQ4HWqQbvMi9QYw',
 'Vile': 'spotify:album:6jgJTpnqytev0AHfkpFKFb',
 'Vile (Expanded Edition)': 'spotify:

In [7]:
# Manually clean some entries, we want originals only and no live albums
albums_to_delete = ['レッド・ビフォー・ブラック', 
                     'Vile (Expanded Edition)', 
                     'The Bleeding - Reissue',
                     'Live Cannibalism',
                     'Torturing And Eviscerating',
                   ]
def get_clean_album_uri_list(artist_albums, albums_to_delete=albums_to_delete):
    for key in albums_to_delete:
        artist_albums.pop(key)  
    artist_albums_uri = [uri for uri in artist_albums.values()]
    return artist_albums_uri

In [8]:
artist_albums_uri = get_clean_album_uri_list(artist_albums, albums_to_delete)
print(artist_albums_uri)

['spotify:album:2Ecgc9nNxJpXd3gboFPbvz', 'spotify:album:3Am2ExqPJjrw51AU2HRJSF', 'spotify:album:7AaZ6zhZXDKilXqsJ52g8b', 'spotify:album:4OZFh833qdZrK6DyelNAwS', 'spotify:album:03jcqKD4ZthRxviJQIPCYf', 'spotify:album:6ilBwRWtEedIQu4Q0YyIux', 'spotify:album:46QEzkj6cg7FxmIHL7zTjR', 'spotify:album:4QnPTzLdkNwnvXxjKZ3AgF', 'spotify:album:4AObEzsEDC6WOAlJCRL9Pa', 'spotify:album:3fEMj3Nr6Dpu7rFvgi1rg9', 'spotify:album:6jgJTpnqytev0AHfkpFKFb', 'spotify:album:1FOBUa0z9RhKPH0frnWMRa', 'spotify:album:2yPAe2RJCfrfV97c8FKKjb']


In [34]:
def get_full_tracklist_dict(artist_albums_uri):
    tracklist = {}
    for album_uri in artist_albums_uri:
        album = sp.album(album_uri)
        for track in album['tracks']['items']:
            tracklist[track['name'].title()] = track['uri']
    return tracklist

In [53]:
full_tracklist = get_full_tracklist_dict(artist_albums_uri)
print(list(full_tracklist.items())[0])
print("Total tracks:", len(full_tracklist))

('Only One Will Die', 'spotify:track:2VDXsHWJ9L0gFm3bL4Ic1Q')
Total tracks: 144


### Audio Features

We use the audio features provided by spotify ([see here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/)) to determine the sonic brutality of a track. We actually only need `Energy`and `Valence` for that, but in addition let's also have a look at the `Dancability`of Cannibal Corpse. Just for fun.

> Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
    
> Valence is a measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). 
    
> Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity.

In [65]:
def get_audio_features_dict(full_tracklist):
    audio_features_dict = {}
    for uri in list(full_tracklist.values()):
        features = sp.audio_features(uri)
        audio_features_dict[uri] = {'energy': features[0]['energy'],
                                    'valcence': features[0]['valence'],
                                    'danceability': features[0]['danceability'],
                                   }
    return audio_features_dict

In [66]:
audio_features_dict = get_audio_features_dict(full_tracklist)
pprint(audio_features_dict)

{'spotify:track:0CP3JuUen1lOo11cUuUr8L': {'danceability': 0.449,
                                          'energy': 0.989,
                                          'valcence': 0.0737},
 'spotify:track:0G074Tus7GLgG5Pr5IpsA0': {'danceability': 0.392,
                                          'energy': 0.997,
                                          'valcence': 0.0579},
 'spotify:track:0JG4Ayqv7WQFcYR2e4Jtt6': {'danceability': 0.263,
                                          'energy': 0.974,
                                          'valcence': 0.325},
 'spotify:track:0PV7aQ8yfo3cr3LLyq7KbV': {'danceability': 0.283,
                                          'energy': 0.99,
                                          'valcence': 0.345},
 'spotify:track:0TkAdjflfnTQniUb8qb7wr': {'danceability': 0.551,
                                          'energy': 0.992,
                                          'valcence': 0.16},
 'spotify:track:0Uh7BOgE78Yc6MZAXraIGP': {'danceability': 0.237,
     

In [68]:
audio_features_dict['spotify:track:0G074Tus7GLgG5Pr5IpsA0']['energy']

0.997

### Create DataFrame for analysis

In [85]:
temp_df1 = pd.DataFrame(full_tracklist.items(), columns = ['title', 'uri'])
temp_df2 = pd.DataFrame(audio_features_dict.items(), columns = ['uri', 'features'])
assert len(temp_df1) == len(temp_df2)

In [90]:
track_df['features'].values[0]['energy']

0.981

In [86]:
track_df = pd.merge(temp_df1, temp_df2, on=['uri'])
track_df

,title,uri,features
0,Only One Will Die,spotify:track:2VDXsHWJ9L0gFm3bL4Ic1Q,"{'energy': 0.981, 'valcence': 0.217, 'danceabi..."
1,Red Before Black,spotify:track:1M3QUZOIhq8LNkMJ8axsLM,"{'energy': 0.994, 'valcence': 0.129, 'danceabi..."
2,Code Of The Slashers,spotify:track:4YWg4PxJzMn84j7Fo32W4M,"{'energy': 0.98, 'valcence': 0.105, 'danceabil..."
3,Shedding My Human Skin,spotify:track:6TlarnloD0Ikiru8CIcg7f,"{'energy': 0.992, 'valcence': 0.0912, 'danceab..."
4,Remaimed,spotify:track:4KlpnaOREJpWNjuoZPeOQ7,"{'energy': 0.998, 'valcence': 0.0397, 'danceab..."
...,...,...,...
139,Necropedophile,spotify:track:5ycOd8aYjGCIrDfLNKqmKW,"{'energy': 0.986, 'valcence': 0.302, 'danceabi..."
140,Entrails Ripped From A Virgin'S Cunt,spotify:track:6GPcwUNmi1zSCyYYPMs3M4,"{'energy': 0.994, 'valcence': 0.238, 'danceabi..."
141,Post Mortal Ejaculation,spotify:track:5w2njyuuFhyCTzGXMTJZo1,"{'energy': 0.993, 'valcence': 0.204, 'danceabi..."
142,Beyond The Cemetery,spotify:track:6WxIMDK9qBfVKVLEu5RVIr,"{'energy': 0.987, 'valcence': 0.206, 'danceabi..."
